In [1]:
# Import SparkSession y la función datediff
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1588582786436_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Create SparkSession Object
spark=SparkSession.builder.appName('data_processing').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Load CSV dataset
df = spark.read.csv('s3://jscardonag/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv', inferSchema = True, header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Borrar las columnas 'Codigo DIVIPOLA', 'fecha reporte web', 'Fecha de muerte'.
df = df.drop('Codigo DIVIPOLA', 'fecha reporte web', 'Fecha de muerte')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Crear la columna donde se almacena el calculo de días que transcurren para que un paciente diagnosticado
# se cure
df = df.withColumn('dias_transcurridos_para_recuperarse', datediff(df['Fecha diagnostico'], df['Fecha recuperado']))
df.filter(df['dias_transcurridos_para_recuperarse'] > 0).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+-------------------+--------------------+-----------------------------------+
|ID de caso|Fecha de notificación|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|       Tipo|Estado| País de procedencia|                 FIS|  Fecha diagnostico|    Fecha recuperado|dias_transcurridos_para_recuperarse|
+----------+---------------------+-------------------+------------------------+----------+----+----+-----------+------+--------------------+--------------------+-------------------+--------------------+-----------------------------------+
|       241|  2020-03-19 00:00:00|        Bogotá D.C.|             Bogotá D.C.|Recuperado|  58|   F|Relacionado|  Leve|            Colombia|2020-03-06T00:00:...|2020-03-23 00:00:00|2020-03-21T00:00:...|                                  2|
|       595|  2020-03-16 00:00:00|          

In [6]:
# Filtrar aquellos pacientes que su 'País de procedencia' = Italia y que su 'Edad' sea mayor o igual a 60 años.
df.filter(df['País de procedencia'] == 'Italia').filter(df['Edad'] >= 59).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+-------------------+------------------------+----------+----+----+---------+------+-------------------+--------------------+-------------------+--------------------+-----------------------------------+
|ID de caso|Fecha de notificación|Ciudad de ubicación|Departamento o Distrito |  atención|Edad|Sexo|     Tipo|Estado|País de procedencia|                 FIS|  Fecha diagnostico|    Fecha recuperado|dias_transcurridos_para_recuperarse|
+----------+---------------------+-------------------+------------------------+----------+----+----+---------+------+-------------------+--------------------+-------------------+--------------------+-----------------------------------+
|        12|  2020-03-10 00:00:00|              Neiva|                   Huila|Recuperado|  74|   F|Importado|  Leve|             Italia|2020-03-06T00:00:...|2020-03-12 00:00:00|2020-04-09T00:00:...|                                -28|
|        16|  2020-03-11 00:00:00|        Bogotá D.C.|  

In [7]:
# Agrupar por fecha de diagnóstico y contar cuantas personas fueron diagnósticadas en esa fecha
df.groupBy('Fecha diagnostico').count().orderBy('count', ascending = False).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|  Fecha diagnostico|count|
+-------------------+-----+
|2020-04-28 00:00:00|  356|
|2020-04-08 00:00:00|  346|
|2020-04-24 00:00:00|  321|
|2020-04-25 00:00:00|  262|
|2020-04-29 00:00:00|  259|
+-------------------+-----+
only showing top 5 rows

In [8]:
# Directorio
write_uri='s3://jscardonag/datasets/spark/df_covid19'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Guardar el dataframe como un solo csv
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Parquet

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Directorio
parquet_uri='s3://jscardonag/datasets/spark/df_covid19_parquet'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Guardar los datos en el formato parquet
df.write.format('parquet').save(parquet_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Attribute name "ID de caso" contains invalid character(s) among " ,;{}()\\n\\t=". Please use alias to rename it.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 738, in save
    self._jwrite.save(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Attribute name "ID de caso" contains invalid character(s) among " ,;{}()\\n\\t=". Please use alias to rename it.;'

